## CNN Text Classification with Keras and Theano

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import _pickle as cPickle # python3 shit
import re, sys, os
os.environ['KERAS_BACKEND']='theano'

In [2]:
from many_stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from itertools import chain
from nltk.classify import NaiveBayesClassifier, accuracy

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model

Using Theano backend.


In [4]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100

In [5]:
data_train = pd.read_csv('dataset/data/text_emotion.csv', sep=',')
print("Dataset shape:",data_train.shape)
print(data_train.sentiment[0],":",data_train.content[0])

Dataset shape: (40000, 4)
empty : @tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =[


In [6]:
stop_words = list(get_stop_words('en'))         #About 900 stopwords
nltk_words = list(stopwords.words('english'))   #About 150 stopwords
stop_words.extend(nltk_words)
 
def remove_stopwords(word_list):
        filtered_tweet=""
        for word in word_list:
            #word = word.lower() 
            if word not in stopwords.words("english"):
                filtered_tweet=filtered_tweet + " " + word
        return filtered_tweet.lstrip()

def clean_tweet( tweet):
        #tweet = tweet.lower() 
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [7]:
def vectorise_label(label):
    if label == "empty":return 1
    elif label == "sadness":return 2
    elif label == "enthusiasm":return 3
    elif label == "neutral":return 4
    elif label == "worry":return 5
    elif label == "surprise":return 6
    elif label == "love":return 7
    elif label == "fun":return 8
    elif label == "hate":return 9
    elif label == "happiness":return 10
    elif label == "boredom":return 11
    elif label == "relief":return 12
    elif label == "anger":return 13
    else:return 0

In [8]:
texts, labels = [], []
print("Creating Lists...")
total = data_train.content.shape[0]
for i in range(data_train.content.shape[0]):
    print("Progress:",round(i/total*100,3),"   ",end="\r")
    texts.append(remove_stopwords(clean_tweet(data_train.content[i])))
    sys.stdout.write("\033[F")
    labels.append(vectorise_label(data_train.sentiment[i]))

Creating Lists...
Progress: 99.997    Progress: 26.905    

In [9]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [10]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 36 unique tokens.


In [11]:
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (40000, 1000)
Shape of label tensor: (40000, 14)


In [12]:
VALIDATION_SPLIT = 0.2

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [20]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of entries in each category:')
print("Training:\n",y_train.sum(axis=0))
print("Validation:\n",y_val.sum(axis=0))

Number of entries in each category:
Training:
 [   0.  638. 4158.  612. 6880. 6796. 1758. 3041. 1414. 1076. 4171.  138.
 1225.   93.]
Validation:
 [   0.  189. 1007.  147. 1758. 1663.  429.  801.  362.  247. 1038.   41.
  301.   17.]


Use pre-trained [GloVe](https://nlp.stanford.edu/projects/glove/) vectors from Stanford NLP. For new words,a "randomised vector" will be created.

In [31]:
GLOVE_DIR = "dataset/glove/glove.840B.300d.txt"
embeddings_index = {}
f = open(GLOVE_DIR)
print("Loading GloVe from",GLOVE_DIR)
for i,line in enumerate(f):
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except Exception as e:
        print("Error:",e)
f.close()
print("Done. Proceeding with Embedding Matrix...")
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print("Creating Embedding Layer...")
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

Loading GloVe from dataset/glove/glove.840B.300d.txt


Progress: 993

Progress: 25505rogress: 1886 2409Progress: 2976Progress: 3545Progress: 4118Progress: 4673Progress: 5262Progress: 5835Progress: 6415Progress: 6969Progress: 7530Progress: 8089Progress: 8643Progress: 9198 9664Progress: 10132 10630Progress: 11204Progress: 11771Progress: 12333Progress: 12897Progress: 13459Progress: 14594 1517015707Progress: 16283 16862 17429 18000 19135Progress: 19691Progress: 20250Progress: 20805Progress: 21354Progress: 21919Progress: 22481Progress: 23551Progress: 24109Progress: 24665Progress: 25214

KeyboardInterrupt: 

Simplified Convolutional structure: total 128 filters with size 5 and max pooling of 5 and 35. [Source](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html).

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')

In [ ]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - simplified convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=128)

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

Deeper Convolutional neural network:In [CNN for Sentence Classification](http://www.aclweb.org/anthology/D14-1181) (Yoon Kim, 2014), multiple filters have been applied. This can be implemented using Keras `Merge` Layer.

In [ ]:
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)

l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)

In [ ]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=20, batch_size=50)

### TODO

 - tune hyper-parameters
 - improve preprocessing
 - use drop-out